In [2]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_case as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)


phases = dr.CLIPS


c:\Users\zhoux\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40, 50],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    # train.Metrics.SDNN, 
    # train.Metrics.MEAN_SCL, 
    # train.Metrics.SCR_RATE, 
# ]
] + train.Metrics.STATISTICAL

In [21]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


random.seed(10)

model_phases = dr.CLIPS

label_type = dr.SelfReports.AROUSAL
threshold = "fixed"

x, y = train.Train_CASE.get_case_data(
    metrics, verbose=False, label_type=label_type, threshold=threshold, 
    normalize=True, combine_phases=False, standardize=True
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.1, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    # features = {name: metrics for name in models.keys()}
    features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # TEST USING OPTIMIZED MODELS AND FEATAURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.73349954 0.74171311 0.74887581 0.74299084 0.74491096 0.7299554
 0.73507044 0.74601287 0.74242842 0.73924173 0.71905819 0.73205998
 0.73464307 0.73713938 0.73555277 0.73173539 0.72071798 0.72509474
 0.73635961 0.73437425 0.73725871 0.72259901 0.73578804 0.74087888
 0.73797784        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}
LGB: {'max_depth': 6, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 30, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 4, 'n_estimators': 30, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [36 12], Predictions: [0], [48]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [36 12], Predictions: [0 1], [34 14]
Training RF ...
Model RF, Actual: [0 1], [36 12], Predictions: [0 1], [32 16]
Training XGB ...
Model XGB, Actual: [0 1], [36 12], Predictions: [0 1], [32 16]

None


[('lf_rr', 260), ('rmssd', 247), ('hf_rr', 212), ('ecg_iqr', 211), ('bpm', 181)]



One or more of the test scores are non-finite: [0.68365858 0.71689527 0.72470132 0.72664494 0.72063118 0.71422997
 0.70434645 0.715842   0.73054868 0.72588395 0.69062343 0.69490099
 0.72008835 0.72818028 0.72484427 0.69012969 0.7049493  0.7199261
 0.73009826 0.72934315 0.6775874  0.68733152 0.70131228 0.70443139
 0.70484992        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
LGB: {'max_depth': 5, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 7, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [38 10], Predictions: [0], [48]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [38 10], Predictions: [0 1], [40  8]
Training RF ...
Model RF, Actual: [0 1], [38 10], Predictions: [0 1], [40  8]
Training XGB ...
Model XGB, Actual: [0 1], [38 10], Predictions: [0 1], [37 11]

None


[('rmssd', 220), ('bpm', 190), ('ecg_iqr', 149), ('hf_rr', 140), ('lf_rr', 91)]




...

Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
LGB: {'max_depth': 6, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 6, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [34 14], Predictions: [0 1], [44  4]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [34 14], Predictions: [0 1], [41  7]
Training RF ...
Model RF, Actual: [0 1], [34 14], Predictions: [0 1], [36 12]
Training XGB ...
Model XGB, Actual: [0 1], [34 14], Predictions: [0 1], [33 15]

None


[('rmssd', 283), ('bpm', 245), ('ecg_iqr', 210), ('hf_rr', 168), ('lf_rr', 110)

One or more of the test scores are non-finite: [0.68665104 0.70603397 0.71410879 0.72253745 0.72799844 0.71630204
 0.72195437 0.7159254  0.72138982 0.72782394 0.71736091 0.71962217
 0.70801895 0.71288698 0.71399374 0.69641602 0.70971761 0.70100726
 0.70238799 0.71011907 0.70212858 0.71070098 0.70643097 0.71208433
 0.71964463        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 4, 'n_estimators': 30, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [34 14], Predictions: [0 1], [46  2]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [34 14], Predictions: [0 1], [37 11]
Training RF ...
Model RF, Actual: [0 1], [34 14], Predictions: [0 1], [37 11]
Training XGB ...
Model XGB, Actual: [0 1], [34 14], Predictions: [0 1], [39  9]

None


[('ecg_iqr', 218), ('bpm', 211), ('rmssd', 208), ('hf_rr', 149), ('lf_rr', 94)]

One or more of the test scores are non-finite: [0.73896937 0.74359977 0.75072412 0.75711424 0.76092209 0.72561273
 0.74409803 0.75227504 0.75458462 0.74905075 0.75873731 0.76980908
 0.76991471 0.76717103 0.76154835 0.73201319 0.75575039 0.76082724
 0.76370154 0.7609431  0.73470635 0.75011396 0.75723988 0.75722278
 0.76055444        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 30, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [28 20], Predictions: [0 1], [43  5]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [28 20], Predictions: [0 1], [40  8]
Training RF ...
Model RF, Actual: [0 1], [28 20], Predictions: [0 1], [36 12]
Training XGB ...
Model XGB, Actual: [0 1], [28 20], Predictions: [0 1], [37 11]

None


[('bpm', 130), ('hf_rr', 93), ('ecg_iqr', 93), ('rmssd', 78), ('lf_rr', 67)]




In [20]:
# ENSEMBLE
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)

from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score

random.seed(42)

test_size = 0.15
folds = 5
ensemble_models = [
    # "SVM", 
    "LGB",
    "RF",
    "XGB"
]

cv, x_train, y_train, x_test, y_test = train.kfold_train_test_split(x, y, test_size, is_resample=False, folds=folds)
y_train = y_train.loc[:, "label"]
y_test = y_test.loc[:, "label"]

estimators = [(name, best_models[name]) for name in ensemble_models]
ensemble = VotingClassifier(estimators, voting="hard")
ensemble.fit(x_train, y_train)
y_pred = ensemble.predict(x_test)

acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, zero_division=0)
r = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_pred)
print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)

	Accuracy: 0.8375
	Precision: 0.7368421052631579
	Recall: 0.6363636363636364
	F1-score: 0.6829268292682926
	AUC score: 0.7750783699059559
----------------------------------------
